### Importing required Libraries

In [65]:
import os
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")

### Converting All text data files into single file

In [66]:
path = "/home/Kapil/anomaly_detection/s3Files/"
os.chdir(path)

In [67]:
final = open("final.txt","a+") 

In [68]:
def read_write_file(file_path):
    with open(file_path, 'r') as f:
        final.write(f.read())

In [69]:
for file in os.listdir():
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
        read_write_file(file_path)

### Reading final text file as pandas dataframe

In [70]:
df = pd.read_csv("/home/bluehole/Kapil/anomaly_detection/s3Files/final.txt",sep=",",index_col=None,header=None)

In [71]:
df.head()

,0,1,2,3,4
0,NaN,1767,2425,384,NaN
1,NaN,1757,2423,757,NaN
2,NaN,1732,2218,759,NaN
3,NaN,1732,2212,765,NaN
4,NaN,1739,2210,769,NaN


### Preprocessing and Feature Selection

In [72]:
df.drop([0,4],axis=1,inplace=True)

In [73]:
df.rename(columns={1:'current',2:'current_signature_analysis',3:'VI_analysis'},inplace=True)

In [74]:
df.head()

,current,current_signature_analysis,VI_analysis
0,1767,2425,384
1,1757,2423,757
2,1732,2218,759
3,1732,2212,765
4,1739,2210,769


In [75]:
df.shape

(4470000, 3)

### Splitting data into train and test set

In [76]:
train = df.iloc[0:3470000]
test = df.iloc[3470000:]

### Anomaly detection

### ISOLATION FOREST

In [77]:
from pycaret.anomaly import *

In [78]:
s = setup(data = train, normalize=True)
iforest = create_model('iforest')

In [79]:
iforest_results = assign_model(iforest)

In [80]:
iforest_results.head()

,current,current_signature_analysis,VI_analysis,Anomaly,Anomaly_Score
0,1767,2425,384,0,-0.056105
1,1757,2423,757,1,0.021140
2,1732,2218,759,0,-0.063526
3,1732,2212,765,0,-0.063360
4,1739,2210,769,0,-0.068516


In [81]:
save_model(iforest, 'saved_iforest_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='UNSUPERVISED_DUMMY_TARGET',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='most frequent',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None...
                 ('fix_perfect', 'passthrough'),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  IFo

In [82]:
saved_iforest = load_model('saved_iforest_model')

Transformation Pipeline and Model Successfully Loaded


In [87]:
pred = predict_model(saved_iforest, data = test)

In [88]:
pred.head()

,current,current_signature_analysis,VI_analysis,Anomaly,Anomaly_Score
3470000,1741,2212,762,0,-0.073564
3470001,1741,2211,760,0,-0.070662
3470002,1742,2209,770,0,-0.074826
3470003,1746,2204,772,0,-0.074633
3470004,1747,1967,822,1,0.028274


In [106]:
train_results_anomaly = iforest_results[iforest_results['Anomaly'] == 1]

In [107]:
train_results_anomaly.head()

,current,current_signature_analysis,VI_analysis,Anomaly,Anomaly_Score
1,1757,2423,757,1,0.021140
40,1933,1918,569,1,0.025109
235,1762,2416,765,1,0.020383
392,2026,1985,373,1,0.023814
510,1948,1971,549,1,0.007325


In [104]:
train_results_anomaly.shape

(173493, 5)

In [108]:
test_results_anomaly = pred[pred['Anomaly'] == 1]

In [109]:
test_results_anomaly.shape

(42747, 5)

In [110]:
# using ISOLATION FOREST
# Total anomaly count in training data 173493
# Total anomaly count in testing data 42747

##  CLUSTERING

In [91]:
cluster = create_model('cluster')

In [92]:
cluster_results = assign_model(cluster)

In [93]:
cluster_results.head()

,current,current_signature_analysis,VI_analysis,Anomaly,Anomaly_Score
0,1767,2425,384,0,0.159209
1,1757,2423,757,1,1.082221
2,1732,2218,759,0,0.187659
3,1732,2212,765,0,0.220387
4,1739,2210,769,0,0.230448


In [94]:
save_model(cluster, 'saved_cluster_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[],
                                       target='UNSUPERVISED_DUMMY_TARGET',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='most frequent',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None...
                 ('fix_perfect', 'passthrough'),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  CBL

In [95]:
saved_cluster = load_model('saved_cluster_model')

Transformation Pipeline and Model Successfully Loaded


In [96]:
pred1 = predict_model(saved_cluster, data = test)

In [97]:
pred1

,current,current_signature_analysis,VI_analysis,Anomaly,Anomaly_Score
3470000,1741,2212,762,0,0.201778
3470001,1741,2211,760,0,0.198508
3470002,1742,2209,770,0,0.235141
3470003,1746,2204,772,0,0.256504
3470004,1747,1967,822,1,0.906612
...,...,...,...,...,...
4469995,1755,2192,774,0,0.310984
4469996,1757,2191,776,0,0.322253
4469997,1763,2189,781,0,0.351203
4469998,1764,2183,779,0,0.370832


In [111]:
train_result = cluster_results[cluster_results['Anomaly'] == 1]

In [112]:
train_result.head()

,current,current_signature_analysis,VI_analysis,Anomaly,Anomaly_Score
1,1757,2423,757,1,1.082221
40,1933,1918,569,1,0.808057
80,2108,2275,103,1,0.954458
81,2102,2282,76,1,0.910366
82,2092,2294,70,1,0.829738


In [113]:
train_result.shape

(173499, 5)

In [114]:
test_result = pred1[pred1['Anomaly'] == 1]

In [115]:
test_result.shape

(45413, 5)

In [116]:
# using CLUSTERING
# Total anomaly count in training data 173499
# Total anomaly count in testing data 45413